# 9. 물체를 분리하자! - 세그멘테이션 살펴보기

그동안 다룬 image classfication은 물체의 종류를 분류하고, object detection은 물체의 존재와 위치를 탐지한다.   
이번에 배워볼 segmentation은 픽셀 수준에서 이미지의 각 부분이 어떤 의미를 갖는지 분리해내는 방법이다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/portrait_mode.max-800x600.jpg)

그림을 보면 segentation 기술을 쉽게 이해할 수 있다.   
쉽게 말하면 __이미지 분할__이다. 사진의 사람 영역과 배경 영역을 분리하고, 배경에 블러처리를 하여 아웃포커싱 효과를 보여주고 있다.

segmentation은 이미지의 영역을 어떤 영역인지 분리하는 기술로 다양한 곳에서 활용되고 있다. 핸드폰의 인물사진 모드가 대표적인 딥러닝 기반 기술이 숨어있는 예시이다.   
이러한 segmentation을 알아보자.

# 9-2. 세그멘테이션 문제의 종류

segmentation 모델들은 다양한 분야에서 활용되고 있다. 자율주행 카메라에 적용되어 이미지를 해석하기도 하고, 의료 이미지에서 병이 있는 영역을 판별해 내기도 한다.

이미지 내에서 영역을 분리하는 접근 방식은 크게 두 가지 방식이 있다. __semantic segmentation__, __instance segmentation__이다.   
영역을 분리하는 관점은 비슷하지만, 접근 방식에 따라 문제의 정의와 모델을 구성하는 방식이 달라진다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/semantic_vs_instance.png)

좌측 상단 사진은 양들이 길에 있는 사진에서 물체들의 localization과 classification을 보는 접근법이다.   
우측 상단은 개별 양들의 개체 하나하나 위치를 정확하게 식별하는 object detection 접근법이다.

좌측 하단은 양, 길, 풀밭의 영역을 정확히 구분해내고 있다. 이러한 접근 방식을 semantic segmentation이라고 한다.   
우측 하단은 양의 영역을 세세하게 개체 단위로 구분하고 있다. 이것을 instance segmentation이라고 한다.

### 1) sementic segmentation
![](https://d3s0tskafalll9.cloudfront.net/media/images/segnet_demo.max-800x600.png)

![](https://d3s0tskafalll9.cloudfront.net/media/images/leaf_segmentation.max-800x600.png)

![](https://d3s0tskafalll9.cloudfront.net/media/images/u-net.max-800x600.png)

위 사진들은 시맨틱 세그멘테이션의 활용 사례들이다.   
그리고 시맨틱 세그멘테이션의 대표적인 모델인 U-Net의 구조이다. 입력으로 572x572 크기의 이미지가 들어가고 출력으로 388x388의 크기에 두 가지 클래스를 가진 segmentation map이 나온다. 이는 가장 마지막 레이어의 채널 개수가 2라는 점에서 확인할 수 있다.

마지막에 나온 두 가지의 클래스를 문제에 따라 다르게 정의하면, 클래스에 따른 sementic segmentation map을 얻을 수 있다.   
인물사진 모드라면 사람과 배경 클래스를, 의료 인공지능에서는 세포 사진에서 병이 있는 영역과 정상인 영역 등을 지정해 적용할 수 있다.

이전에 다뤄봤던 모델들과 비교하면 input, output의 이미지 scale이 비교적 큰데, 이는 segmentation을 위해 각 픽셀에 해당하는 영역의 클래스별 정보가 필요하기 때문이다.

### 2) instance segmentation
인스턴스 세그멘테이션은 같은 클래스 내에서도 각 개체(instance)들을 분리하여 세그멘테이션을 수행한다. 하지만 어떻게 접근하면 좋을까?   
object detection 모델로 각 개체를 구분하고 이후에 개체별 sementic segmentation을 수행하면 괜찮지 않을까?

이러한 방식 중 대표적인게 Mask R-CNN이다. 2017년에 발표한 Mask-R-CNN은 2-stage object detection의 가장 대표적인 Faster-R-CNN을 계승했다.

Faster-R-CNN의 아이디어인 Region of Interest Pooling Layer(RoIPool) 개념을 개선하여 정확한 segmentation에 유리하게 한 RolAlign와 클래스별 마스크 분리라는 두 가지 아이디어를 적용했다.   
이는 클래스별 object detection과 sementic segmentation을 하나의 task로 엮어낸 것으로 평가받는 중요한 모델이다.

Faster-R-CNN 계열에서 사용하는 RoIPool 개념과 그 한계점에 대해 알아보자.   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/GC-5-L-FasterRCNN-01.png)

RoIPool Layer는 다양한 RoI 영역을 Pooling을 통해 동일한 크기의 feature map으로 추출하는 레이어이다. 이후 이 고정 사이즈의 feature map을 바탕으로 바운딩 박스와 object 클래스를 추론한다.

이 구성은 object detection에서는 크게 문제가 없다. 하지만 object 영역의 정확한 마스킹을 필요하는 sementation에서 문제가 된다.   

![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-5-L-RoIPool-01.max-800x600.png)

RoIPool 과정에서 Quantization이 필요하다는 점이다. 그림의 예시에서는 16x16으로 분할했다. 이미지에서 RoI 영역의 크기는 다양한데, 모든 RoI 영역의 가로, 세로 픽셀의 크기가 16배수인 것은 아니다.   
그래서 가로 200, 세로 145 픽셀짜리 RoI 영역을 16x16으로 분할된 영역 중 절반 이상이 RoI 영역에 덮이는 곳들로 끼워 맞추다 보면, 영역 밖이 포함되는 경우도 있고, 자투리 영역이 버려지는 경우도 생긴다.   
이는 시맨틱 세그멘테이션의 정보손실과 왜곡을 야기하게 된다.   
위 그림에서 3x3 RoIPool을 적용했을 때 초록색은 불필요한 정보가 들어있고, 하늘색과 파란색은 정보를 잃어버리게 된다.

Mask-R-CNN의 RoIAlign은 RoI를 처리할 고정 사이즈의 Feature map을 생성할 수 있게 아이디어를 제공한다.   
__핵심 아이디어__는 RoI 영역을 pooling layer 크기에 맞춰 등분한 후, RoIPool을 했을 때 quantization 영역 중 가까운 것들과 bilinear interpolation 계산을 통해 생성해야 할 feature map을 계산해낸다는 점이다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/instance_seg.max-800x600.png)

Mask R-CNN은 Faster R-CNN에서 추출방식을 RoIAlign 방식으로 개선을하고 세그멘테이션을 더한 방식이다. 사진의 Faster R-CNN과 비교한 구조도를 보면 U-Net처럼 feature map의 크기를 키워 mask를 생성하는 부분을 통해 인스턴스에 해당하는 영역인 인스턴스 맵을 추론한다.   
Mask R-CNN은 클래스에 따른 mask를 예측할 때, 여러 가지 task를 한 모델로 학습하여 물체 검출의 성능을 높인다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/mask-rcnn-head.max-800x600.png)

그림을 보면 bounding box regression을 하는 Bbox head와 마스크를 예측하는 Mask Head 두 갈래로 나뉘고 있다.   
Mask map의 경우 시맨틱 세그멘테이션과 달리 상대적으로 작은 28x28의 특성 맵 크기를 갖는다. RoIAlign을 통해 줄어든 특성에서 마스크를 예측하기 때문에 정확한 마스크를 얻으려는 경우에는 부적합할 수 있다.

# 9-3. 주요 세그멘테이션 모델(1) FCN

# 9-4. 주요 세그멘테이션 모델(2) U-Net

# 9-5. 주요 세그멘테이션 모델(3) DeepLab 계열

# 9-6. 세그멘테이션의 평가

# 9-7. Upsampling의 다양한 방법